In [1]:
import sys
sys.path.append('..')
from src.data.docx  import read_docx,process_docx,find_content
import glob
import json
import os
import pandas as pd
import ast
from functools import reduce
from statistics import mode
import re
from nltk.corpus import stopwords
import spacy
from gensim.utils import simple_preprocess

from gensim.models.phrases import Phrases, Phraser
from gensim import corpora


In [2]:
'''Defining CONSTS'''
docx_path = '../data/raw/docx'
groundbase_dir = '../data/raw/groundbase'
transcripts_dir = os.path.join(groundbase_dir,'transcripts')
topic_dataset_path = os.path.join(groundbase_dir,'dataset.csv')
transcript_filespath = glob.glob(groundbase_dir + '/transcripts/*.json')
videos_ids = list(map(lambda fl: fl.split('\\')[-1].split('.')[0],glob.glob(docx_path + '/*')))

desired_videos =['7kLHJ-F33GI'] #['zWg7U0OEAoE','tORLeHHtazM'] #['7kLHJ-F33GI','RIawrYLVdIw','7snJ1mx1EMQ']
videos_ids = list(filter(lambda x: x in desired_videos,videos_ids))

f_read = {
    'statbook':'statbook',
    'Dsa':'Dsa'
}

docxs_chapter_keyword = {
    'statbook':'Topic',
    'Dsa': 'Chapter'
}



In [31]:
video_to_paper = {}
paper_content ={}
for vid in videos_ids:
    doc_path = glob.glob(os.path.join(docx_path,vid + '/*.docx'))[0]
    doc_name = doc_path.split('\\')[-1].split('.')[0]
    video_to_paper[vid] = doc_name    
    if doc_name not in paper_content.keys():    
        full_text,font_sizes = read_docx(doc_path)
        paper_content[doc_name] = find_content(f_read[doc_name],full_text,font_sizes,main_chapter_keyword=docxs_chapter_keyword[doc_name])

In [32]:
'''Find the following for each paper:
    1) main chapter title index. for example [10,15....]
    2) range of subsection within each main chapter . for example [(0,9),(11,14)...]
    
    '''

paper_mainchapter_indexes = {}
paper_sec_within_main_indexes = {}
paper_mains_as_one_doc = {}
paper_subsec_as_one_doc = {}


for doc_name in paper_content.keys():
    '''Find the main chapter indexes in the list of the overall titles'''
    mainchapter_indexes = [paper_content[doc_name]['titles'].index(ch_title)
                           for ch_title in paper_content[doc_name]['main titles']]
    '''Find the subsection indexes range within each main chapter '''
    subsec_mainchapter_indexes = [range(mainchapter_indexes[index],mainchapter_indexes[index + 1])
                                  for index in range(len(mainchapter_indexes) - 1)]
    subsec_mainchapter_indexes.append(range(mainchapter_indexes[-1],
                                            len(paper_content[doc_name]['titles'])))
    paper_mainchapter_indexes[doc_name] = mainchapter_indexes
    paper_sec_within_main_indexes[doc_name] = subsec_mainchapter_indexes
    
    
    '''Making each chapter as a one documents'''
    
    '''Union all the documents in a section into single document'''
    paper_subsec_as_one_doc[doc_name] = [list(reduce(lambda doc,acc:doc + acc,sec,[]))
                                         for sec in paper_content[doc_name]['corpus']]
    '''Union all the sub section in a main chapter into one document'''
    paper_mains_as_one_doc[doc_name] = [list(reduce(lambda acc,s_i:
                                                    paper_subsec_as_one_doc[doc_name][s_i]+acc,subsec_indexes,[]))
                                        for subsec_indexes in paper_sec_within_main_indexes[doc_name]]


In [33]:
bigram = Phrases(paper_mains_as_one_doc['statbook'], min_count=5, threshold=50) # higher threshold fewer phrases.
bigram_mod = Phraser(bigram)
documents_bigrams = [bigram_mod[doc] for doc in paper_mains_as_one_doc['statbook']] 

#[[print(w) for w in d if '_' in w] for d in documents_bigrams]
trigram = Phrases(bigram[paper_mains_as_one_doc['statbook']], threshold=3)
print(trigram[bigram[paper_mains_as_one_doc['statbook'][14]]])

['answer_select_exercise', 'word', 'increase', 'decrease', 'maximum', 'would', 'like', 'maximize', 'likelihood', 'give', 'number_recapture', 'individual', 'domain', 'nonnegative', 'integer', 'can', 'use', 'calculus', 'look', 'ratio', 'likelihood', 'value', 'successive', 'value', 'total_population', 'ratio', 'large', 'computation', 'show', 'ratio', 'great', 'small', 'value', 'less', 'large', 'value', 'place', 'maximum', 'expand', 'binomial', 'coefficient', 'expression', 'njr', 'simplify', 'technical', 'aspect', 'use', 'concept', 'introduce', 'obtain', 'property', 'maximum_likelihood_estimator', 'choice', 'estimator', 'desirable', 'property', 'maximum', 'estimator', 'question', 'arise', 'would', 'choose', 'method_moment_estimator', 'answer', 'use', 'maximum_likelihood', 'technique', 'rely', 'know', 'density_function', 'form', 'density', 'must', 'amenable', 'analysis', 'find', 'fisher_information', 'less', 'experiment', 'need', 'order', 'compute', 'moment', 'compute', 'consider', 'case', 

In [17]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()


def myprocess_docx(total_corpus):
    total_corpus_tokenized =[]

    for chapter_corpus in total_corpus:
        start_index = 0
        documents = []
        for m in re.finditer('[a-zA-Z]\.',chapter_corpus):
            chapter_tokenized_doc = chapter_corpus[start_index:m.start(0) + 1]
            
            doc_text_no_punc = simple_preprocess(chapter_tokenized_doc,deacc=True) 
            tokenized_text_non_stop_words = [ word for word in doc_text_no_punc \
                                             if word not in stop_words]
            
            '''Form bigram and trigram'''
            """bigram = Phrases(tokenized_text_non_stop_words, min_count=5, threshold=100) # higher threshold fewer phrases.
            bigram_mod = Phraser(bigram)
            documents_bigrams = [bigram_mod[doc] for doc in tokenized_text_non_stop_words] 
            trigram = Phrases(bigram[tokenized_text_non_stop_words], threshold=3)
            trigram_documents = [trigram[bigram[doc]] for doc in tokenized_text_non_stop_words]"""
            
            #text_non_stop_words = ' '.join(tokenized_text_non_stop_words)
            
            
            
            """tokenized_lemmas = nlp(text_non_stop_words)
            tokenized_lemmas = [token.lemma_ for token in tokenized_lemmas \
                                if token.pos_ in allowed_postags]"""
            
            tokenized_stem = [porter.stem(w) for w in tokenized_text_non_stop_words]
            
            
            documents.append(tokenized_stem)
            start_index = m.end(0) + 1
        total_corpus_tokenized.append(documents)
    total_corpus_tokenized = [ [doc for doc in ch if len(doc) > 0 ] for ch in total_corpus_tokenized]
    return total_corpus_tokenized

In [25]:
stop_words = stopwords.words('english')
nlp = spacy.load('en',disable=['parser','ner'])
allowed_postags=['NOUN', 'ADJ', 'VERB']

def myfind_content_statbook(full_text,font_sizes,main_chapter_keyword = 'Topic'):
    most_common_font_size = mode(font_sizes)
    topic_titles = []
    main_chapter_titles = []
    p_Start_Section = re.compile('{} \d?'.format(main_chapter_keyword))
    p_subsection_numbering = re.compile(r'^((\d+\.)+\d*)$')
    p_words = re.compile('[A-Za-z]+')
    
    main_chapter_indexes = [i+1 for i in range(len(full_text) - 1)\
                            if p_Start_Section.match(full_text[i]) is not None and font_sizes[i] > most_common_font_size]
    sub_chapter_indexes  = [i for i in range(len(full_text))\
                            if p_subsection_numbering.match(full_text[i]) is not None
                            and
                            font_sizes[i] > most_common_font_size]
    all_titles_indexes = main_chapter_indexes + sub_chapter_indexes    
    all_titles_indexes.sort()
    
    
    '''Finding titles values'''
    topic_titles  = []
    for j in all_titles_indexes:
        # subsection
        if j not in main_chapter_indexes:
           # if no label is given while reading the doc
           # ..
           # 2.1.2
           # my corpus and my content
            if '.' in full_text[j + 1] or ',' in full_text[j + 1]:
                topic_titles.append(full_text[j])
            else:
                # 1.1
                # introduction
                topic_titles.append("%s %s" %(full_text[j],full_text[j+1]))
        else:
            # main chapter
            topic_titles.append(full_text[j])

    main_chapter_titles = [full_text[j] for j in main_chapter_indexes]

    
    
    total_corpus  = []
    for i in range(len(all_titles_indexes) - 1):
        sec_as_text = reduce(lambda acc,x: acc+" " +x,
                             full_text[all_titles_indexes[i]:all_titles_indexes[i+1]],"")
        # adding dot for finishing the corpus
        if len(sec_as_text) == 0:
            pass
            #sec_as_text = 'section empty.' # planting plain text for non empty
        elif sec_as_text[-1] != '.':
            sec_as_text = sec_as_text + '.'
        total_corpus.append(sec_as_text)
        
    
    # adding the text of the last section
    last_sec_text = ""
    i = all_titles_indexes[-1] + 1
    while i < len(full_text) and font_sizes[i] <= most_common_font_size:
        last_sec_text = last_sec_text + " " + full_text[i]
        i+=1
    total_corpus.append(last_sec_text)
    
    
    # fill empty subssections with random string
     
    total_corpus = myprocess_docx(total_corpus)
    
    for i in range(len(total_corpus)):
        if len(total_corpus[i]) == 0:
            total_corpus[i] = [['section','empty']]
    
    
    
    
    
    return {'corpus':total_corpus,'titles':topic_titles,'main titles':main_chapter_titles}
    #return total_corpus

In [26]:
video_to_paper = {}
paper_content ={}
for vid in [videos_ids[0]]:
    doc_path = glob.glob(os.path.join(docx_path,vid + '/*.docx'))[0]
    doc_name = doc_path.split('\\')[-1].split('.')[0]
    video_to_paper[vid] = doc_name    
    if doc_name not in paper_content.keys():    
        full_text,font_sizes = read_docx(doc_path)
        paper_content[doc_name] = myfind_content_statbook(full_text,font_sizes)

In [27]:
print(len(corpus))
print(paper_content['statbook']['corpus'][116])

198
[['cramer', 'rao', 'bound', 'topic', 'somewhat', 'advanc', 'skip', 'first', 'read'], ['section', 'give', 'us', 'introduct', 'log', 'likelihood', 'deriv', 'score', 'function'], ['shall', 'encount', 'function', 'introduc', 'maximum', 'likelihood', 'estim'], ['addit', 'cramer', 'rao', 'bound', 'base', 'varianc', 'score', 'function', 'known', 'fisher', 'inform', 'give', 'lower', 'bound', 'varianc', 'unbias', 'estim'], ['concept', 'necessari', 'describ', 'varianc', 'maximum', 'likelihood', 'estim'], ['among', 'unbias', 'estim', 'one', 'import', 'goal', 'find', 'estim', 'small', 'varianc', 'possibl', 'precis', 'goal', 'would', 'find', 'unbias', 'estim', 'uniform', 'minimum', 'varianc'], ['word', 'effici', 'unbias', 'estim', 'minimum', 'valu', 'ratio', 'var', 'var', 'valu', 'thu', 'effici', 'goal', 'find', 'estim', 'effici', 'near', 'one', 'possibl'], ['unbias', 'estim', 'cramer', 'rao', 'bound', 'tell', 'us', 'small', 'varianc', 'ever', 'possibl'], ['formula', 'bit', 'mysteri', 'first'],

In [28]:
'''Find the following for each paper:
    1) main chapter title index. for example [10,15....]
    2) range of subsection within each main chapter . for example [(0,9),(11,14)...]
    
    '''

paper_mainchapter_indexes = {}
paper_sec_within_main_indexes = {}
paper_mains_as_one_doc = {}
paper_subsec_as_one_doc = {}


for doc_name in paper_content.keys():
    '''Find the main chapter indexes in the list of the overall titles'''
    mainchapter_indexes = [paper_content[doc_name]['titles'].index(ch_title)
                           for ch_title in paper_content[doc_name]['main titles']]
    '''Find the subsection indexes range within each main chapter '''
    subsec_mainchapter_indexes = [range(mainchapter_indexes[index],mainchapter_indexes[index + 1])
                                  for index in range(len(mainchapter_indexes) - 1)]
    subsec_mainchapter_indexes.append(range(mainchapter_indexes[-1],
                                            len(paper_content[doc_name]['titles'])))
    paper_mainchapter_indexes[doc_name] = mainchapter_indexes
    paper_sec_within_main_indexes[doc_name] = subsec_mainchapter_indexes
    
    
    '''Making each chapter as a one documents'''
    
    '''Union all the documents in a section into single document'''
    paper_subsec_as_one_doc[doc_name] = [list(reduce(lambda doc,acc:doc + acc,sec,[]))
                                         for sec in paper_content[doc_name]['corpus']]
    '''Union all the sub section in a main chapter into one document'''
    paper_mains_as_one_doc[doc_name] = [list(reduce(lambda acc,s_i:
                                                    paper_subsec_as_one_doc[doc_name][s_i]+acc,subsec_indexes,[]))
                                        for subsec_indexes in paper_sec_within_main_indexes[doc_name]]


In [30]:
bigram = Phrases(paper_mains_as_one_doc['statbook'], min_count=5, threshold=50) # higher threshold fewer phrases.
bigram_mod = Phraser(bigram)
documents_bigrams = [bigram_mod[doc] for doc in paper_mains_as_one_doc['statbook']] 

#[[print(w) for w in d if '_' in w] for d in documents_bigrams]
trigram = Phrases(bigram[paper_mains_as_one_doc['statbook']], threshold=3)
print(trigram[bigram[paper_mains_as_one_doc['statbook'][14]]])

['answer_select_exercis', 'word', 'ln', 'pjx', 'increas', 'decreas', 'thu', 'maximum', 'would', 'like', 'maxim_likelihood', 'given', 'number_recaptur', 'individu', 'domain', 'nonneg', 'integ', 'cannot', 'use', 'calculu', 'howev', 'look', 'ratio', 'likelihood', 'valu', 'success', 'valu', 'total_popul', 'njr', 'jr', 'like', 'precis', 'ratio', 'larger', 'one', 'comput', 'show', 'ratio', 'greater', 'small', 'valu', 'less', 'one', 'larg', 'valu', 'thu', 'place', 'middl', 'maximum', 'expand', 'binomi', 'coeffici', 'express', 'njr', 'simplifi', 'technic', 'aspect', 'use', 'concept', 'previous', 'introduc', 'obtain', 'properti', 'maximum_likelihood_estim', 'choic', 'estim', 'desir', 'properti', 'maximum_likelihood_estim', 'question', 'aris', 'would', 'one', 'choos', 'method_moment_estim', 'one', 'answer', 'use', 'maximum_likelihood', 'techniqu', 'reli', 'know', 'densiti_function', 'explicitli', 'moreov', 'form', 'densiti', 'must', 'amen', 'analysi', 'necessari', 'maxim_likelihood', 'find', 'fi